In [1]:
import pandas as pd

In [2]:
train_df= pd.read_csv('../input/arkon-data-bikes-train/train_set.csv')
test_df= pd.read_csv('../input/arkon-data-bikes-test/test_set.csv')

/opt/conda/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3552: DtypeWarning: Columns (8) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


# 2.- Data Preparation

In [3]:
train_df.dtypes

trip_id                  int64
duration                 int64
start_time              object
end_time                object
start_lat              float64
start_lon              float64
end_lat                float64
end_lon                float64
bike_id                 object
plan_duration          float64
trip_route_category     object
passholder_type         object
start_station            int64
end_station              int64
dtype: object

In [4]:
train_df.dtypes

trip_id                  int64
duration                 int64
start_time              object
end_time                object
start_lat              float64
start_lon              float64
end_lat                float64
end_lon                float64
bike_id                 object
plan_duration          float64
trip_route_category     object
passholder_type         object
start_station            int64
end_station              int64
dtype: object

In [5]:
train_df.isnull().sum()

trip_id                    0
duration                   0
start_time                 0
end_time                   0
start_lat               5563
start_lon               5563
end_lat                18574
end_lon                18574
bike_id                    0
plan_duration            208
trip_route_category        0
passholder_type         2576
start_station              0
end_station                0
dtype: int64

In [6]:
test_df.isnull().sum()

trip_id                    0
duration                   0
start_time                 0
end_time                   0
start_lat               4622
start_lon               4622
end_lat                14891
end_lon                14891
bike_id                    0
trip_route_category        0
start_station              0
end_station                0
dtype: int64

In [7]:
print(len(train_df.columns))
print(len(test_df.columns))

14
12


Como es posible observar, en ambos conjuntos de datos (train_set y test_set) existen valores nulos en diferentes columnas. Para este problema existen dos opciones de solución: eliminar los ejemplos con datos faltantes o "rellenarlos" (***Data Imputation*** ) con nuevos valores.

Eliminar los datos se presenta como la opción más rápida y sencilla. Sin embargo, esto implicaría la perdida de ejemplos que probablemente afectarían el rendimiento de cualquier modelo.

Por lo anterior se decidió utilizar ***Data Imputation*** a partir del cálculo de la media aritmética con el objetivo de evitar el problema descrito previamente y simplificar el proceso de aplicación de esta técnica.

Sin embargo, es necesario eliminar 2576 ejemplos del conjunto de entrenamiento debido a que no esta presnete el valor de 'passholder_type'.

In [8]:
# Review columns
NoMatch_columns = []
for column in train_df.columns:
    for index, column2 in enumerate(test_df.columns):
        if column == column2:
            break
        elif index + 1 == len(test_df.columns):
            NoMatch_columns.append(column)
NoMatch_columns

['plan_duration', 'passholder_type']

In [9]:
train_df.isnull().sum()

trip_id                    0
duration                   0
start_time                 0
end_time                   0
start_lat               5563
start_lon               5563
end_lat                18574
end_lon                18574
bike_id                    0
plan_duration            208
trip_route_category        0
passholder_type         2576
start_station              0
end_station                0
dtype: int64

### passholder_type

In [10]:
len(train_df)

700000

In [11]:
train_df = train_df.dropna(subset = ['passholder_type'])

In [12]:
len(train_df)

697424

### plan_duration

In [13]:
train_df.drop('plan_duration', inplace = True, axis = 1)

### start_time and end_time

In [14]:
columns = ['start_time', 'end_time']
for column in columns:
    train_df[column] = pd.to_datetime(train_df[column])
    test_df[column] = pd.to_datetime(test_df[column])

In [15]:
train_df['start_time'].head(6)

0   2018-08-07 11:20:00
1   2017-09-17 17:51:00
2   2019-04-22 09:22:00
3   2019-09-22 11:27:00
4   2020-01-31 17:11:00
5   2017-12-16 15:18:00
Name: start_time, dtype: datetime64[ns]

In [16]:
test_df['start_time'].head(6)

0   2017-01-01 00:24:00
1   2017-01-01 00:28:00
2   2017-01-01 00:39:00
3   2017-01-01 00:43:00
4   2017-01-01 00:56:00
5   2017-01-01 01:54:00
Name: start_time, dtype: datetime64[ns]

### trip_id

In [17]:
train_df.drop('trip_id', inplace = True, axis = 1)
test_df.drop('trip_id', inplace = True, axis = 1)

### start_lat, start_lon, end_lat, end_lon

In [18]:
dict_temp = {'start_lat':0, 'start_lon':0, 'end_lat':0, 'end_lon':0}

for column in dict_temp.keys():
    dict_temp[column] = train_df[column].mean()
    train_df[column] = train_df[column].fillna(dict_temp[column])
    test_df[column] = test_df[column].fillna(dict_temp[column])

### bike_id

In [19]:
train_df.drop('bike_id', inplace = True, axis = 1)
test_df.drop('bike_id', inplace = True, axis = 1)

### trip_route_category

In [20]:
train_df["trip_route_category"]

0         Round Trip
1            One Way
2            One Way
3            One Way
4            One Way
             ...    
699995       One Way
699996       One Way
699997       One Way
699998       One Way
699999       One Way
Name: trip_route_category, Length: 697424, dtype: object

In [21]:
pd.get_dummies(train_df["trip_route_category"])

,One Way,Round Trip
0,0,1
1,1,0
2,1,0
3,1,0
4,1,0
...,...,...
699995,1,0
699996,1,0
699997,1,0
699998,1,0


In [22]:
for df in [train_df, test_df]:
    temp_df = pd.get_dummies(df["trip_route_category"])
    for column in temp_df.columns.tolist():
        df.insert(loc = len(df.columns), column = column, value = temp_df[column])
    df.drop("trip_route_category", inplace = True, axis = 1)

### passholder_type

Será modifcado en la siguiente libreta dinámica

## Make DataFrame with means of each column train_df

In [23]:
means_dict = {}
for column in train_df.columns:
    if column == 'passholder_type':
        continue
    if column in dict_temp.keys():
        means_dict[column] = [dict_temp[column]]
        print(column)
    else:
        means_dict[column] = [train_df[column].mean()]
print(means_dict)
print()
df_means = pd.DataFrame.from_dict(means_dict)
df_means

start_lat
start_lon
end_lat
end_lon
{'duration': [37.12179104820024], 'start_time': [Timestamp('2019-04-07 06:28:18.260835840')], 'end_time': [Timestamp('2019-04-07 07:12:24.123294976')], 'start_lat': [34.04494252410217], 'start_lon': [-118.25364788320726], 'end_lat': [34.044163510779796], 'end_lon': [-118.25900137054006], 'start_station': [3498.2580008717796], 'end_station': [3488.2667702860813], 'One Way': [0.8315687444079929], 'Round Trip': [0.16843125559200717]}



,duration,start_time,end_time,start_lat,start_lon,end_lat,end_lon,start_station,end_station,One Way,Round Trip
0,37.121791,2019-04-07 06:28:18.260835840,2019-04-07 07:12:24.123294976,34.044943,-118.253648,34.044164,-118.259001,3498.258001,3488.26677,0.831569,0.168431


In [24]:
len(means_dict.keys())

11

In [25]:
train_df.dtypes

duration                    int64
start_time         datetime64[ns]
end_time           datetime64[ns]
start_lat                 float64
start_lon                 float64
end_lat                   float64
end_lon                   float64
passholder_type            object
start_station               int64
end_station                 int64
One Way                     uint8
Round Trip                  uint8
dtype: object

In [26]:
test_df.dtypes

duration                  int64
start_time       datetime64[ns]
end_time         datetime64[ns]
start_lat               float64
start_lon               float64
end_lat                 float64
end_lon                 float64
start_station             int64
end_station               int64
One Way                   uint8
Round Trip                uint8
dtype: object

In [27]:
train_df.isnull().sum()

duration           0
start_time         0
end_time           0
start_lat          0
start_lon          0
end_lat            0
end_lon            0
passholder_type    0
start_station      0
end_station        0
One Way            0
Round Trip         0
dtype: int64

In [28]:
train_df.isnull().sum()

duration           0
start_time         0
end_time           0
start_lat          0
start_lon          0
end_lat            0
end_lon            0
passholder_type    0
start_station      0
end_station        0
One Way            0
Round Trip         0
dtype: int64

In [29]:
print(len(train_df.columns))
print(len(test_df.columns))

12
11


In [30]:
train_df.head()

,duration,start_time,end_time,start_lat,start_lon,end_lat,end_lon,passholder_type,start_station,end_station,One Way,Round Trip
0,35,2018-08-07 11:20:00,2018-08-07 11:55:00,33.748920,-118.275192,33.748920,-118.275192,Walk-up,4127,4127,0,1
1,32,2017-09-17 17:51:00,2017-09-17 18:23:00,34.035679,-118.270813,34.047749,-118.243172,Walk-up,3057,3062,1,0
2,6,2019-04-22 09:22:00,2019-04-22 09:28:00,34.046070,-118.233093,34.047749,-118.243172,Monthly Pass,3022,3062,1,0
3,138,2019-09-22 11:27:00,2019-09-22 13:45:00,34.062580,-118.290092,34.059689,-118.294662,One Day Pass,4304,4311,1,0
4,14,2020-01-31 17:11:00,2020-01-31 17:25:00,34.026291,-118.277687,34.021660,-118.278687,Monthly Pass,4266,4443,1,0


In [31]:
test_df.head()

,duration,start_time,end_time,start_lat,start_lon,end_lat,end_lon,start_station,end_station,One Way,Round Trip
0,12,2017-01-01 00:24:00,2017-01-01 00:36:00,34.058319,-118.246094,34.058319,-118.246094,3028,3028,0,1
1,17,2017-01-01 00:28:00,2017-01-01 00:45:00,34.049980,-118.247162,34.043732,-118.260139,3027,3018,1,0
2,20,2017-01-01 00:39:00,2017-01-01 00:59:00,34.063389,-118.236160,34.044159,-118.251579,3066,3055,1,0
3,12,2017-01-01 00:43:00,2017-01-01 00:55:00,34.048851,-118.246422,34.050140,-118.233238,3029,3079,1,0
4,48,2017-01-01 00:56:00,2017-01-01 01:44:00,34.049198,-118.252831,34.049198,-118.252831,3063,3063,0,1


## Save Files

In [32]:
train_df.to_csv('train_set_final.csv', index=False)
test_df.to_csv('test_set_final.csv', index=False)
df_means.to_csv('means_dict.csv', index=False)